In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# need to fix total df because each week should hold agg stats from only prev weeks, not including itself
# keep fixing mismattches with the weeks when making final df

In [2]:
df2 = pd.read_csv("../data/game_stats.csv")
df2.head()

,Unnamed: 0,id,home.school,home.conference,home.points,home.fumblesRecovered,home.rushingTDs,home.puntReturnYards,home.puntReturnTDs,home.puntReturns,home.passingTDs,home.kickingPoints,home.firstDowns,home.thirdDownEff,home.fourthDownEff,home.totalYards,home.netPassingYards,home.completionAttempts,home.yardsPerPass,home.rushingYards,home.rushingAttempts,home.yardsPerRushAttempt,home.totalPenaltiesYards,home.turnovers,home.fumblesLost,home.interceptions,home.possessionTime,away.school,away.conference,away.points,away.fumblesRecovered,away.rushingTDs,away.puntReturnYards,away.puntReturnTDs,away.puntReturns,away.passingTDs,away.kickingPoints,away.interceptionYards,away.interceptionTDs,away.passesIntercepted,away.firstDowns,away.thirdDownEff,away.fourthDownEff,away.totalYards,away.netPassingYards,away.completionAttempts,away.yardsPerPass,away.rushingYards,away.rushingAttempts,away.yardsPerRushAttempt,away.totalPenaltiesYards,away.turnovers,away.fumblesLost,away.interceptions,away.possessionTime,Year,Week,home.interceptionYards,home.interceptionTDs,home.passesIntercepted,away.kickReturnYards,away.kickReturnTDs,away.kickReturns,home.kickReturnYards,home.kickReturnTDs,home.kickReturns,home.totalFumbles,home.tacklesForLoss,home.defensiveTDs,home.tackles,home.sacks,home.qbHurries,home.passesDeflected,away.totalFumbles,away.tacklesForLoss,away.defensiveTDs,away.tackles,away.sacks,away.qbHurries,away.passesDeflected
0,0,242410259,Virginia Tech,ACC,13,0.0,0.0,10.0,0.0,1.0,1.0,7.0,18.0,5-13,0-0,294.0,180.0,14-29,5.3,141.0,34.0,4.1,0-0,3.0,2.0,1.0,19:30,USC,Pac-10,24,2.0,0.0,60.0,0.0,4.0,3.0,6.0,32.0,0.0,1.0,16.0,6-14,0-1,373.0,284.0,19-29,9.4,101.0,29.0,3.5,0-0,0.0,0.0,0.0,17:08,2004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,242410193,Miami (OH),Mid-American,49,2.0,3.0,116.0,1.0,7.0,3.0,7.0,26.0,4-11,1-3,454.0,292.0,24-37,7.9,162.0,39.0,4.2,0-0,3.0,2.0,1.0,18:05,Indiana State,NaN,0,2.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,12.0,0-0,0-0,204.0,137.0,18-33,4.8,44.0,31.0,1.4,8-63,3.0,2.0,1.0,18:28,2004,1,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,242482426,Navy,FBS Independents,27,0.0,3.0,30.0,0.0,5.0,1.0,3.0,23.0,4-10,3-4,430.0,134.0,8-9,14.3,301.0,57.0,5.3,0-0,3.0,3.0,0.0,18:43,Duke,ACC,12,3.0,1.0,NaN,NaN,NaN,0.0,6.0,NaN,NaN,NaN,14.0,2-13,1-2,265.0,115.0,13-22,4.1,175.0,39.0,4.5,0-0,0.0,0.0,0.0,17:57,2004,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,242480120,Maryland,ACC,23,1.0,2.0,22.0,0.0,2.0,0.0,11.0,21.0,5-12,0-0,367.0,198.0,12-22,7.7,198.0,46.0,4.3,0-0,4.0,3.0,1.0,18:15,Northern Illinois,Mid-American,20,3.0,0.0,-4.0,0.0,1.0,1.0,4.0,1.0,0.0,1.0,20.0,4-18,2-2,337.0,228.0,19-37,5.9,118.0,38.0,3.1,0-0,3.0,1.0,2.0,18:34,2004,2,36.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,242480153,North Carolina,ACC,49,0.0,5.0,47.0,0.0,5.0,2.0,7.0,23.0,10-12,0-0,575.0,236.0,14-24,9.8,341.0,45.0,7.6,0-0,4.0,3.0,1.0,17:39,William & Mary,NaN,38,3.0,0.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,21.0,0-0,0-0,442.0,322.0,23-41,7.9,120.0,31.0,3.9,5-45,0.0,0.0,0.0,19:01,2004,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def process_third(x):
    if(str(x).split('-')[-1] == '0'): return 1
    else: return float(str(x).split('-')[0])/float(str(x).split('-')[1])

stats = df2[['id', 'home.school','home.points', 'home.thirdDownEff','home.completionAttempts', 'home.yardsPerPass','home.yardsPerRushAttempt','home.totalPenaltiesYards','home.turnovers','home.possessionTime','away.school','away.points','away.thirdDownEff','away.completionAttempts', 'away.yardsPerPass','away.yardsPerRushAttempt','away.totalPenaltiesYards','away.turnovers','away.possessionTime','home.rushingAttempts','away.rushingAttempts','Year','Week']]
stats.dropna(inplace = True)
stats.loc[:,'home.pointDiff'] = df2['home.points'] - df2['away.points']
stats.loc[:,'away.pointDiff'] = -stats['home.pointDiff']
stats.loc[:,'home.completionAttempts'] = df2['home.completionAttempts'].apply(lambda x: float(str(x).split('-')[-1]))
stats.loc[:,'away.completionAttempts'] = df2['away.completionAttempts'].apply(lambda x: float(str(x).split('-')[-1]))
stats.loc[:,'home.yardsAllowedPerPlay'] = df2['away.totalYards']/(stats['away.completionAttempts'] + df2['away.rushingAttempts'])
stats.loc[:,'away.yardsAllowedPerPlay'] = df2['home.totalYards']/(stats['home.completionAttempts'] + df2['home.rushingAttempts'])
stats.loc[:,'home.yards_per_play'] = stats['away.yardsAllowedPerPlay']
stats.loc[:,'away.yards_per_play'] = stats['home.yardsAllowedPerPlay']
stats.loc[:,'home.forcedTO'] = df2['away.turnovers']
stats.loc[:,'away.forcedTO'] = df2['home.turnovers']
stats.loc[:,'home.passAttemptsAllowed'] = stats['away.completionAttempts']
stats.loc[:,'away.passAttemptsAllowed'] = stats['home.completionAttempts']
stats.loc[:,'home.yardsAllowedPerRush'] = df2['away.yardsPerRushAttempt']
stats.loc[:,'away.yardsAllowedPerRush'] = df2['home.yardsPerRushAttempt']
stats.loc[:,'home.thirdDownEff'] = stats.loc[:,'home.thirdDownEff'].apply(process_third)
stats.loc[:,'away.thirdDownEff'] = stats.loc[:,'away.thirdDownEff'].apply(process_third)
stats.loc[:,'home.totalPenaltiesYards'] = stats.loc[:,'home.totalPenaltiesYards'].apply(lambda x: float(x.split('-')[-1]))
stats.loc[:,'away.totalPenaltiesYards'] = stats.loc[:,'away.totalPenaltiesYards'].apply(lambda x: float(x.split('-')[-1]))
stats.loc[:,'home.possessionTime'] = stats['home.possessionTime'].apply(lambda x: float(x.split(':')[0])*60 + float(x.split(':')[1]))
stats.loc[:,'away.possessionTime'] = stats['away.possessionTime'].apply(lambda x: float(x.split(':')[0])*60 + float(x.split(':')[1]))
stats['home.win'] = (stats['home.points'] > stats['away.points']).astype(int)
stats['away.win'] = (stats['away.points'] > stats['home.points']).astype(int)
stats.iloc[1239,22] = 10
stats = stats.drop(9284, axis = 0)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

In [4]:
away_index = [x for x in list(stats.columns) if 'away.' in x] + ['Week', 'Year', 'id']
away = stats[away_index].drop('away.points', axis = 1)
away.columns = [x.split('.')[-1] for x in list(away.columns)]
away['isHome'] = 0
away = away.groupby(['Year', 'school','Week']).mean()
away

thirdDownEff  completionAttempts  yardsPerPass  \
Year school           Week                                                   
2004 Air Force        4         0.375000                14.0           9.9   
                      5         0.523810                16.0          10.6   
                      10        0.538462                21.0           9.0   
                      11        0.416667                 8.0          10.5   
     Akron            2         0.384615                36.0           6.2   
...                                  ...                 ...           ...   
2020 West Virginia    8         0.266667                50.0           6.9   
     Western Kentucky 2         0.333333                23.0           5.6   
                      5         0.411765                36.0           5.2   
                      7         0.083333                35.0           4.6   
     Wyoming          8         0.187500                32.0           7.3   

                            yardsPerRushAttempt  totalPenaltiesYards  \
Year school           Week                                             
2004 Air Force        4                     3.9                  0.0   
                      5                     4.6                  0.0   
                      10                    4.8                  0.0   
                      11                    5.2                  0.0   
     Akron            2                     2.3                  0.0   
...                                         ...                  ...   
2020 West Virginia    8                     3.1                 69.0   
     Western Kentucky 2                     3.4                 23.0   
                      5                     3.6                 55.0   
                      7                     2.8                 39.0   
     Wyoming          8                     3.7                  6.0   

                            turnovers  possessionTime  rushingAttempts  \
Year school           Week                                               
2004 Air Force        4           0.0          1454.0             71.0   
                      5           3.0          1273.0             66.0   
                      10          2.0           783.0             48.0   
                      11          0.0          1184.0             66.0   
     Akron            2           3.0          1034.0             31.0   
...                               ...             ...              ...   
2020 West Virginia    8           1.0          1765.0             29.0   
     Western Kentucky 2           1.0          1647.0             35.0   
                      5           1.0          1895.0             38.0   
                      7           4.0          1713.0             32.0   
     Wyoming          8           1.0          1502.0             35.0   

                            pointDiff  yardsAllowedPerPlay  yards_per_play  \
Year school           Week                                                   
2004 Air Force        4          17.0             5.058824        4.905882   
                      5         -14.0             7.318841        5.743902   
                      10        -17.0             6.352273        6.086957   
                      11          9.0             5.878378        5.797297   
     Akron            2         -38.0             8.515625        4.402985   
...                               ...                  ...             ...   
2020 West Virginia    8          -7.0             5.194030        5.544304   
     Western Kentucky 2         -14.0             6.763889        4.275862   
                      5           3.0             4.833333        4.405405   
                      7         -23.0             5.424242        3.731343   
     Wyoming          8          -3.0             6.280000        5.388060   

                            forcedTO  passAttemptsAllowed  \
Year school           Week           

In [5]:
home_index = [x for x in list(stats.columns) if 'home.' in x] + ['Week', 'Year', 'id']
home = stats[home_index].drop('home.points',axis = 1)
home.columns = [x.split('.')[-1] for x in list(home.columns)]
home['isHome'] = 1
home = home.groupby(['Year', 'school','Week']).mean()

In [6]:
total = pd.concat([home,away]).groupby(['Year', 'school','Week']).mean()
total.iloc[-10:,:]

thirdDownEff  completionAttempts  yardsPerPass  \
Year school           Week                                                   
2020 West Virginia    7         0.375000                44.0           7.2   
                      8         0.266667                50.0           6.9   
     Western Kentucky 2         0.333333                23.0           5.6   
                      3         0.300000                25.0           7.7   
                      5         0.411765                36.0           5.2   
                      6         0.166667                39.0           5.4   
                      7         0.083333                35.0           4.6   
                      8         0.250000                25.0           5.3   
     Wisconsin        8         0.571429                21.0          11.8   
     Wyoming          8         0.187500                32.0           7.3   

                            yardsPerRushAttempt  totalPenaltiesYards  \
Year school           Week                                             
2020 West Virginia    7                     5.9                 76.0   
                      8                     3.1                 69.0   
     Western Kentucky 2                     3.4                 23.0   
                      3                     3.5                 19.0   
                      5                     3.6                 55.0   
                      6                     3.4                  0.0   
                      7                     2.8                 39.0   
                      8                     3.7                 25.0   
     Wisconsin        8                     3.4                 15.0   
     Wyoming          8                     3.7                  6.0   

                            turnovers  possessionTime  rushingAttempts  \
Year school           Week                                               
2020 West Virginia    7           2.0          1975.0             38.0   
                      8           1.0          1765.0             29.0   
     Western Kentucky 2           1.0          1647.0             35.0   
                      3           0.0          1501.0             28.0   
                      5           1.0          1895.0             38.0   
                      6           3.0          1519.0             25.0   
                      7           4.0          1713.0             32.0   
                      8           2.0          1615.0             33.0   
     Wisconsin        8           1.0          2608.0             54.0   
     Wyoming          8           1.0          1502.0             35.0   

                            pointDiff  yardsAllowedPerPlay  yards_per_play  \
Year school           Week                                                   
2020 West Virginia    7          21.0             2.907407        6.634146   
                      8          -7.0             5.194030        5.544304   
     Western Kentucky 2         -14.0             6.763889        4.275862   
                      3          -6.0             6.164557        5.490566   
                      5           3.0             4.833333        4.405405   
                      6         -24.0             5.548387        4.593750   
                      7         -23.0             5.424242        3.731343   
                      8           3.0             3.461538        4.379310   
     Wisconsin        8          38.0             4.804348        5.733333   
     Wyoming          8          -3.0             6.280000        5.388060   

                            forcedTO  passAttemptsAllowed  \
Year school           Week                                  
2020 West Virginia    7          2.0                 23.0   
                      8          1.0                 28.0   
     Western Kentucky 2          2.0                 34.0   
                      3          0.0                 21.0   
                      5          0

In [7]:
idx = list(total.index)
idx.reverse()
n = 1
ids = total['id']
field_status = total['isHome']
outcomes = total['win']
total.drop(['id','isHome'], axis = 1, inplace = True) 
todrop = []
for i,x in enumerate(idx[:-1]):
    if(x[1] != idx[i + 1][1]):
        todrop.append(x)
        continue
    j = 2
    agg = total.loc[idx[i+1],:]
    while((i + j < len(idx)) and x[1] == idx[i + j][1]):
        agg = agg + total.loc[idx[i+j],:]
        j += 1
        n += 1
    total.loc[x,:] = agg/n
    n = 1;
total['id'] = ids
total['isHome'] = field_status
total['won_game'] = outcomes
total.drop(todrop, axis = 0, inplace = True)

In [8]:
# need to drop the rows in todrop, put stats back in matchup df from unique gameids in total
rowlist = []
for gameID in stats['id']:
    if len(total[total['id'] == gameID]) != 2: continue
    hometeam = list((total[(total['id'] == gameID) & (total['isHome'] == 1)].values)[0])
    awayteam = list((total[(total['id'] == gameID) & (total['isHome'] == 0)].values)[0])
    rowlist.append(hometeam + awayteam)
    rowlist.append(awayteam + hometeam)
matchups = pd.DataFrame(rowlist, columns = (list(total.columns) + ['other.' + x for x in list(total.columns)])).drop(['other.id','other.isHome','other.won_game'], axis = 1)
matchups = matchups.set_index('id')
matchups['win_pct_diff'] = matchups['win'] - matchups['other.win']
matchups.drop(['win', 'other.win'], axis = 1, inplace = True)
outcome = matchups['won_game']
matchups.drop('won_game', axis = 1, inplace = True)
matchups['outcome'] = outcome

In [9]:
# import everything
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.svm import SVC


In [10]:
# Pull out X and y, scale data
matchups = matchups.astype(float).dropna()
X = matchups.iloc[:,:-1]
y = matchups.iloc[:,-1]
scaler = preprocessing.StandardScaler().fit(X)
X_t = scaler.transform(X)

In [24]:
### Logistic Classifier --- Best 10-fold mean crossval Score = .7068545320687714

# X_train, X_test, y_train, y_test = train_test_split(matchups.iloc[:,:-1], matchups.iloc[:,-1],test_size = .25, random_state = 42)
clf = LogisticRegression(penalty = 'l1', solver = 'saga').fit(X_t, y)
# grid = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
# logreg_cv = GridSearchCV(clf, grid, cv=10)
# logreg_cv.fit(X_t,y)
# best = cross_val_score(clf, X_t, y, cv=10).mean()
michigan_week1 = [.5,25,9,8.3,80,1,24*60 + 51, 31,25,4.5, 8.6,4,31, 3.1, 1]
michigan_st_week1 = [.418, 31,7.4, 1.3, 52, 9, 28*60 + 57, 39, -11, 276/73, 369/85, 3, 30, 2.6, 1]
game = np.array(michigan_week1 + michigan_st_week1)
game_t = scaler.transform(game.reshape(1,-1))
print("prediction = {} with a probability of {}".format(clf.predict(game_t)[0], clf.predict_proba(game_t)[0][-1]))

prediction = 1.0 with a probability of 0.9446168843535756


In [40]:
# Support Vector Machine -- 
grid = {"C":np.logspace(-3,3,7), 'kernel':['linear', 'poly', 'rbf', 'sigmoid']}
clf = SVC()
svc_cv = GridSearchCV(clf, grid, cv=5)
svc_cv.fit(X_t, y)
print(svc_cv.best_params_)
# cross_val_score(clf, X_t, y, cv=5).mean()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarni

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarni

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarni

KeyboardInterrupt: 